In [1]:
import math
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import random
import statistics
import lightgbm as lgb

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv(r"c:\work\dataset\rosbank\train.csv", ",", decimal=".")
train["is_test"] = False
test = pd.read_csv(r"c:\work\dataset\rosbank\test.csv", ",", decimal=".")
test["is_test"] = True
atm_data = test.append(train)
atm_data.columns = ["original_index", "address", "address_rus", "atm_group", "id", "is_test", "lat", "long", "target"]

atm_data.shape

(8765, 9)

In [3]:
counts = atm_data.groupby('address_rus')['id'].count().reset_index().rename(columns={'id':'count'})
atm_data = pd.merge(atm_data, counts, how='left', on='address_rus')

In [4]:
geo_data = pd.read_csv(r"c:\work\dataset\rosbank\near_atm.csv", ",", engine='python')
geo_data = geo_data.loc[:, ["atm", "dist", "place", "category"]].sort_values("dist")
geo_data.head(5)

,atm,dist,place,category
56038,22,0.214,mall,shop
117760,2221,0.305,mall,shop
219372,235,0.310,mall,shop
31547,8369,0.323,department_store,shop
13948,8370,0.323,department_store,shop


In [5]:
levels = pd.read_csv(r"c:\work\dataset\rosbank\levels.csv", ",", engine='python', decimal=".",
                     usecols=["atm", "levels", "dist"])
levels["levels"] = pd.to_numeric(levels["levels"], errors='coerce')
levels["dist"] = pd.to_numeric(levels["dist"], errors='coerce')
levels = levels.loc[:,["atm", "levels", "dist"]].sort_values("dist")
levels.head()

,atm,levels,dist
115899,3669,6.0,0.102
147201,3668,6.0,0.102
184899,588,6.0,0.102
107668,6713,3.0,0.141
181958,4657,3.0,0.190


In [6]:
cities = pd.read_csv(r"c:\work\dataset\rosbank\cities.csv", ";", engine='python')
cities = cities.loc[:, ["Индекс", "Тип региона", "Регион", "Город", "Уровень по ФИАС", 
                        "Признак центра района или региона", "Часовой пояс", "Широта", "Долгота", 
                        "Федеральный округ", "Население"]]
cities.dropna(subset =["Индекс"], axis=0, inplace = True)
cities.reset_index(inplace=True)
cities["Население"] = pd.to_numeric(cities["Население"], errors='coerce')

In [7]:
cities.head()

,index,Индекс,Тип региона,Регион,Город,Уровень по ФИАС,Признак центра района или региона,Часовой пояс,Широта,Долгота,Федеральный округ,Население
0,0,385200.0,Респ,Адыгея,Адыгейск,4: город,0,UTC+3,44.878372,39.190172,Южный,12689.0
1,1,385000.0,Респ,Адыгея,Майкоп,4: город,2,UTC+3,44.609827,40.100653,Южный,144055.0
2,2,649000.0,Респ,Алтай,Горно-Алтайск,4: город,2,UTC+7,51.958268,85.960296,Сибирский,62861.0
3,3,658125.0,край,Алтайский,Алейск,4: город,0,UTC+7,52.492091,82.779415,Сибирский,28528.0
4,4,656000.0,край,Алтайский,Барнаул,4: город,2,UTC+7,53.348115,83.779836,Сибирский,635585.0


In [8]:
# Function for selection of city closest to atm
def find_closest_city(lat, long, what):

    #Due to low calculation speed we narrow square of finding closest city. This gives obout 100x speedup
    c1 = cities["Широта"]< (lat + 2)
    c2 = cities["Широта"]> (lat - 2)
    c3 = cities["Долгота"]> (long - 2)
    c4 = cities["Долгота"]< (long + 2)
    partial = cities[c1 & c2 & c3 & c4]                           
    
    if len(partial)>0:
        i = partial.apply(lambda x: (math.pow(lat - x["Широта"], 2) + math.pow(long - x["Долгота"],2)), axis=1).idxmin()
    else:
        i = cities.apply(lambda x: (math.pow(lat - x["Широта"], 2) + math.pow(long - x["Долгота"],2)), axis=1).idxmin()
    if (math.isnan(i)):
        return None
    else:
        return  cities.loc[cities.index[i], what]

# Source table has duplicated records with same index but different city
atm_data["Индекс"]=atm_data.apply(lambda x: (find_closest_city( x["lat"], x["long"], "Индекс")), axis=1)
atm_data["Город"]=atm_data.apply(lambda x: (find_closest_city( x["lat"], x["long"], "Город")), axis=1)
atm_data.shape

(8765, 12)

In [9]:
atm_data = pd.merge(atm_data, cities, on=["Индекс", "Город"], how="left")

counts2 = atm_data.groupby('Город')['id'].count().reset_index().rename(columns={'id':'count2'})
atm_data = pd.merge(atm_data, counts2, how='left', on='Город')


atm_data =atm_data.loc[:,['original_index', 'atm_group', 'id',
       'is_test', 'lat', 'long', 'target', 'Тип региона', 'Уровень по ФИАС',
       'Признак центра района или региона', 'Широта',
       'Долгота', 'Федеральный округ', 'Население', 'Индекс', 'Город', 'count', 'count2']]
atm_data.columns = ['original_index', 'atm_group', 'atm',
       'is_test', 'lat', 'long', 'target', 'region_type', 'levelFIAS',
       'status', 'city_center_lat',
       'city_center_long', 'federal', 'population', 'post_index', 'city', 'count_on_same_address' , 'count_in_same_city']

In [10]:
atm_data

,original_index,atm_group,atm,is_test,lat,long,target,region_type,levelFIAS,status,city_center_lat,city_center_long,federal,population,post_index,city,count_on_same_address,count_in_same_city
0,0,5478.0,8096.0,True,53.757824,87.159786,NaN,обл,4: город,0.0,53.794276,87.214405,Сибирский,547885.0,654000.0,Новокузнецк,2.0,57.0
1,1,3185.5,2570.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
2,2,3185.5,2428.0,True,52.918598,158.514038,NaN,край,4: город,0.0,52.930428,158.405728,Дальневосточный,22905.0,684090.0,Вилючинск,1.0,9.0
3,3,3185.5,2799.0,True,55.533496,89.178148,NaN,край,4: город,0.0,55.538968,89.180156,Сибирский,38570.0,662310.0,Шарыпово,1.0,5.0
4,4,1942.0,250.0,True,55.179216,61.297729,NaN,обл,4: город,2.0,55.160366,61.400786,Уральский,1130273.0,454000.0,Челябинск,3.0,105.0
5,5,5478.0,5759.0,True,54.971827,73.411646,NaN,обл,4: город,2.0,54.984814,73.367464,Сибирский,1154000.0,644000.0,Омск,4.0,193.0
6,6,5478.0,8274.0,True,55.856295,37.493919,NaN,обл,4: город,0.0,55.888670,37.430415,Центральный,207125.0,141400.0,Химки,3.0,64.0
7,7,5478.0,6797.0,True,51.652908,39.292059,NaN,обл,4: город,2.0,51.659238,39.196828,Центральный,889680.0,394000.0,Воронеж,1.0,93.0
8,8,3185.5,2218.0,True,55.028269,73.250039,NaN,обл,4: город,2.0,54.984814,73.367464,Сибирский,1154000.0,644000.0,Омск,1.0,193.0
9,9,5478.0,8465.0,True,51.689197,39.269637,NaN,обл,4: город,2.0,51.659238,39.196828,Центральный,889680.0,394000.0,Воронеж,1.0,93.0


In [11]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6378.137 #Earth radius in km
    dLat = lat2 * math.pi / 180 - lat1 * math.pi / 180
    dLon = lon2 * math.pi / 180 - lon1 * math.pi / 180
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180) \
        * math.cos(lat2 * math.pi / 180) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d * 1000

atm_data["dist_to_center"] = atm_data.apply(
    lambda x : haversine(x["lat"], x["long"], x["city_center_lat"], x["city_center_long"]), axis=1)

In [12]:
knc = KNeighborsClassifier()
dots = atm_data[['lat','long']].fillna(-1)
knc.fit(X=dots , y=np.ones(dots.shape[0]))
distances, indexes = knc.kneighbors(X=dots, n_neighbors=6)

for i in range(1,6):
    atm_data["dist_to_closest_atm%s"%i] = distances[:,i]
    if (i==1):
        avg=distances[:,i]
    else:
        avg = avg + distances[:,i]
    
avg = avg/5
atm_data["dist_to_closest_atm_average"] = avg

In [13]:
def atm_in_range(lat, long):
    #Due to low calculation speed we narrow square of finding closest atm.
    c1 = atm_data["lat"]< (lat + 2)
    c2 = atm_data["lat"]> (lat - 2)
    c3 = atm_data["long"]> (long - 2)
    c4 = atm_data["long"]< (long + 2)
    partial = atm_data[c1 & c2 & c3 & c4]                           
    
    if len(partial)>0:
        i = partial.apply(lambda x: 1 if haversine(x["lat"], x["long"], lat, long)<500 else 0, axis=1).sum()
    else:
        i = atm_data.apply(lambda x: 1 if haversine(x["lat"], x["long"], lat, long)<500 else 0, axis=1).sum()
    if (math.isnan(i)):
        return 10000
    else:
        return  i
    
atm_data["home_atm_in_range"] = atm_data.apply(lambda x : atm_in_range(x["lat"], x["long"]), axis=1)

In [14]:
atm_data.head()

,original_index,atm_group,atm,is_test,lat,long,target,region_type,levelFIAS,status,city_center_lat,city_center_long,federal,population,post_index,city,count_on_same_address,count_in_same_city,dist_to_center,dist_to_closest_atm1,dist_to_closest_atm2,dist_to_closest_atm3,dist_to_closest_atm4,dist_to_closest_atm5,dist_to_closest_atm_average,home_atm_in_range
0,0,5478.0,8096.0,True,53.757824,87.159786,NaN,обл,4: город,0.0,53.794276,87.214405,Сибирский,547885.0,654000.0,Новокузнецк,2.0,57.0,5419.895573,0.000000,0.014118,0.014118,0.015074,0.021648,0.012992,2
1,1,3185.5,2570.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2,2,3185.5,2428.0,True,52.918598,158.514038,NaN,край,4: город,0.0,52.930428,158.405728,Дальневосточный,22905.0,684090.0,Вилючинск,1.0,9.0,7387.129365,0.006418,0.006418,0.108855,0.108855,0.109296,0.067968,3
3,3,3185.5,2799.0,True,55.533496,89.178148,NaN,край,4: город,0.0,55.538968,89.180156,Сибирский,38570.0,662310.0,Шарыпово,1.0,5.0,622.174851,0.005297,0.012029,0.016566,0.125603,0.689073,0.169714,1
4,4,1942.0,250.0,True,55.179216,61.297729,NaN,обл,4: город,2.0,55.160366,61.400786,Уральский,1130273.0,454000.0,Челябинск,3.0,105.0,6880.126968,0.000000,0.000000,0.005201,0.006717,0.006717,0.003727,4


In [15]:
objects_in_range =  lambda g: g.count()
closest_place =  lambda g: g.iloc[0]
shops_in_range = lambda g: g[g=="shop"].count() 
tourism_in_range = lambda g: g[g=="tourism"].count() 
amenity_in_range = lambda g: g[g=="amenity"].count() 
public_transport_in_range = lambda g: g[g=="public_transport"].count() 
banks_in_range = lambda g: g[g.str.contains("bank")].count()
atm_in_range = lambda g: g[g.str.contains("atm")].count()
large_shops_in_range = lambda g: g[g=="mall"].count() + g[g=="supermarket"].count()
market_in_range = lambda g: g[g=="marketplace"].count()
dist_to_closest_parking = lambda g: geo_data.loc[g[g=="parking"].index]["dist"].min()
dist_to_closest_transport = lambda g: geo_data.loc[g[g=="public_transport"].index]["dist"].min()
dist_to_closest_tourism = lambda g: geo_data.loc[g[g=="tourism"].index]["dist"].min()
dist_to_closest_shop = lambda g: geo_data.loc[g[g=="shop"].index]["dist"].min()
dist_to_closest_object = lambda g: geo_data.loc[g.index]["dist"].min()
dist_to_closest_place_cafe = lambda g: geo_data.loc[g[g=="cafe"].index]["dist"].min()
dist_to_closest_place_clinic = lambda g: geo_data.loc[g[g=="clinic"].index]["dist"].min()
dist_to_closest_place_convenience = lambda g: geo_data.loc[g[g=="convenience"].index]["dist"].min()
dist_to_closest_place_department_store = lambda g: geo_data.loc[g[g=="department_store"].index]["dist"].min()
dist_to_closest_place_hospital = lambda g: geo_data.loc[g[g=="hospital"].index]["dist"].min()
dist_to_closest_place_hotel = lambda g: geo_data.loc[g[g=="hotel"].index]["dist"].min()
dist_to_closest_place_kindergarten = lambda g: geo_data.loc[g[g=="kindergarten"].index]["dist"].min()
dist_to_closest_place_kiosk = lambda g: geo_data.loc[g[g=="kiosk"].index]["dist"].min()
dist_to_closest_place_mall = lambda g: geo_data.loc[g[g=="mall"].index]["dist"].min()
dist_to_closest_place_marketplace = lambda g: geo_data.loc[g[g=="marketplace"].index]["dist"].min()
dist_to_closest_place_place_of_worship = lambda g: geo_data.loc[g[g=="place_of_worship"].index]["dist"].min()
dist_to_closest_place_public_building = lambda g: geo_data.loc[g[g=="public_building"].index]["dist"].min()
dist_to_closest_place_school = lambda g: geo_data.loc[g[g=="school"].index]["dist"].min()
dist_to_closest_place_supermarket = lambda g: geo_data.loc[g[g=="supermarket"].index]["dist"].min()
dist_to_closest_place_townhall = lambda g: geo_data.loc[g[g=="townhall"].index]["dist"].min()
dist_to_closest_place_university = lambda g: geo_data.loc[g[g=="university"].index]["dist"].min()

f = {
    'category':
        {
            "objects_in_range" : objects_in_range,
            "closest_category" : closest_place, 
            "shops_in_range" : shops_in_range,
            "tourism_in_range" : tourism_in_range,
            "amenity_in_range" : amenity_in_range,
            "public_transport_in_range" : public_transport_in_range,
            "dist_to_closest_transport" : dist_to_closest_transport,
            "dist_to_closest_tourism" : dist_to_closest_tourism,
            "dist_to_closest_shop" : dist_to_closest_shop,
            "dist_to_closest_object" : dist_to_closest_object
        }, 
    'place':
        {
            "closest_place" : closest_place, 
            "banks_in_range" : banks_in_range, 
            "atm_in_range" : atm_in_range,
            "dist_to_closest_parking": dist_to_closest_parking,
            "large_shops_in_range" : large_shops_in_range,
            "market_in_range" : market_in_range,
            "dist_to_closest_place_cafe" : dist_to_closest_place_cafe,
            "dist_to_closest_place_clinic" : dist_to_closest_place_clinic,
            "dist_to_closest_place_convenience" : dist_to_closest_place_convenience,
            "dist_to_closest_place_department_store" : dist_to_closest_place_department_store,
            "dist_to_closest_place_hospital" : dist_to_closest_place_hospital,
            "dist_to_closest_place_hotel" : dist_to_closest_place_hotel,
            "dist_to_closest_place_kindergarten" : dist_to_closest_place_kindergarten,
            "dist_to_closest_place_kiosk" : dist_to_closest_place_kiosk,
            "dist_to_closest_place_mall" : dist_to_closest_place_mall,
            "dist_to_closest_place_marketplace" : dist_to_closest_place_marketplace,
            "dist_to_closest_place_place_of_worship" : dist_to_closest_place_place_of_worship,
            "dist_to_closest_place_public_building" : dist_to_closest_place_public_building,
            "dist_to_closest_place_school" : dist_to_closest_place_school,
            "dist_to_closest_place_supermarket" : dist_to_closest_place_supermarket,
            "dist_to_closest_place_townhall" : dist_to_closest_place_townhall,
            "dist_to_closest_place_university" : dist_to_closest_place_university
        }
    }


gdata = geo_data.groupby('atm').agg(f)
gdata.columns = gdata.columns.droplevel(0)
gdata['atm'] = gdata.index
gdata["dist_to_closest_transport"].fillna(1000, inplace = True)
gdata["dist_to_closest_parking"].fillna(1000, inplace = True)
gdata["dist_to_closest_tourism"].fillna(1000, inplace = True)
gdata["dist_to_closest_shop"].fillna(1000, inplace = True)
gdata["dist_to_closest_place_cafe"].fillna(1000, inplace = True)
gdata["dist_to_closest_place_clinic"].fillna(1000, inplace = True)
gdata["dist_to_closest_place_convenience"].fillna(1000, inplace = True)
gdata["dist_to_closest_place_department_store"].fillna(1000, inplace = True)
gdata["dist_to_closest_place_hospital"].fillna(1000, inplace = True)
gdata["dist_to_closest_place_hotel"].fillna(1000, inplace = True)
gdata["dist_to_closest_place_kindergarten"].fillna(1000, inplace = True)
gdata["dist_to_closest_place_kiosk"].fillna(1000, inplace = True)
gdata["dist_to_closest_place_mall"].fillna(1000, inplace = True)
gdata["dist_to_closest_place_marketplace"].fillna(1000, inplace = True)
gdata["dist_to_closest_place_place_of_worship"].fillna(1000, inplace = True)
gdata["dist_to_closest_place_public_building"].fillna(1000, inplace = True)
gdata["dist_to_closest_place_school"].fillna(1000, inplace = True)
gdata["dist_to_closest_place_supermarket"].fillna(1000, inplace = True)
gdata["dist_to_closest_place_townhall"].fillna(1000, inplace = True)
gdata["dist_to_closest_place_university"].fillna(1000, inplace = True)
gdata.head(5)

,objects_in_range,closest_category,shops_in_range,tourism_in_range,amenity_in_range,public_transport_in_range,dist_to_closest_transport,dist_to_closest_tourism,dist_to_closest_shop,dist_to_closest_object,closest_place,banks_in_range,atm_in_range,dist_to_closest_parking,large_shops_in_range,market_in_range,dist_to_closest_place_cafe,dist_to_closest_place_clinic,dist_to_closest_place_convenience,dist_to_closest_place_department_store,dist_to_closest_place_hospital,dist_to_closest_place_hotel,dist_to_closest_place_kindergarten,dist_to_closest_place_kiosk,dist_to_closest_place_mall,dist_to_closest_place_marketplace,dist_to_closest_place_place_of_worship,dist_to_closest_place_public_building,dist_to_closest_place_school,dist_to_closest_place_supermarket,dist_to_closest_place_townhall,dist_to_closest_place_university,atm
atm,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,64,shop,10,6,30,18,296.170,156.814,156.607,156.607,mall,0,0,353.692,6,1,363.229,218.944,1000.000,1000.000,1000.000,242.245,1000.000,1000.0,156.607,276.926,185.255,1000.000,369.498,1000.000,1000.000,213.934,1
2,43,amenity,3,0,36,4,295.951,1000.000,308.718,70.468,parking,0,0,70.468,3,0,432.003,1000.000,1000.000,1000.000,323.495,1000.000,369.353,1000.0,308.718,1000.000,278.656,222.027,342.008,527.977,1000.000,476.765,2
3,41,amenity,3,0,36,2,354.608,1000.000,192.216,129.170,parking,0,0,129.170,2,0,1000.000,430.546,1000.000,1000.000,529.283,1000.000,288.705,1000.0,298.626,1000.000,1000.000,1000.000,266.450,312.845,1000.000,1000.000,3
4,42,shop,13,4,25,0,1000.000,256.694,62.051,62.051,clothes,1,0,408.445,1,1,304.614,1000.000,278.165,512.836,1000.000,440.089,1000.000,1000.0,1000.000,393.871,1000.000,220.909,67.974,282.834,111.063,184.810,4
5,20,shop,7,0,13,0,1000.000,1000.000,31.293,31.293,supermarket,0,0,36.748,1,2,132.512,1000.000,310.042,1000.000,349.868,1000.000,283.265,1000.0,1000.000,128.068,1000.000,1000.000,484.573,31.293,1000.000,1000.000,5


In [16]:
closest_levels = lambda g: g.iloc[0]
average_levels = lambda g: g.mean()
max_levels = lambda g: g.max()
dist_to_closest_building =  lambda g: levels.loc[g.index]["dist"].min()
dist_to_closest_high_building =  lambda g: levels.loc[g[g >=9].index]["dist"].min()
count_of_buildings = lambda g: g.count()
average_levels_of_closest_buildings =  lambda g: levels.loc[g[g <100].index]["levels"].mean()

f2 = {
    'levels':
        {
            "closest_levels" : closest_levels,
            "average_levels" : average_levels, 
            "max_levels" : max_levels,
            "dist_to_closest_building" : dist_to_closest_building,
            "dist_to_closest_high_building" : dist_to_closest_high_building,
            "count_of_buildings" : count_of_buildings,
        }, 
    'dist':
        {
            "average_levels_of_closest_buildings" : average_levels_of_closest_buildings, 
        }
    }
ldata = levels.groupby('atm').agg(f2)
ldata.columns = ldata.columns.droplevel(0)
ldata['atm'] = ldata.index
ldata["dist_to_closest_high_building"].fillna(1000, inplace = True)

In [17]:
atm_data = pd.merge(atm_data, gdata, on=["atm"], how="left")
atm_data = pd.merge(atm_data, ldata, on=["atm"], how="left")
atm_data.to_csv(r"c:\work\dataset\rosbank\atm_features.csv",";", index=None)

In [18]:
atm_data.head()

,original_index,atm_group,atm,is_test,lat,long,target,region_type,levelFIAS,status,city_center_lat,city_center_long,federal,population,post_index,city,count_on_same_address,count_in_same_city,dist_to_center,dist_to_closest_atm1,dist_to_closest_atm2,dist_to_closest_atm3,dist_to_closest_atm4,dist_to_closest_atm5,dist_to_closest_atm_average,home_atm_in_range,objects_in_range,closest_category,shops_in_range,tourism_in_range,amenity_in_range,public_transport_in_range,dist_to_closest_transport,dist_to_closest_tourism,dist_to_closest_shop,dist_to_closest_object,closest_place,banks_in_range,atm_in_range,dist_to_closest_parking,large_shops_in_range,market_in_range,dist_to_closest_place_cafe,dist_to_closest_place_clinic,dist_to_closest_place_convenience,dist_to_closest_place_department_store,dist_to_closest_place_hospital,dist_to_closest_place_hotel,dist_to_closest_place_kindergarten,dist_to_closest_place_kiosk,dist_to_closest_place_mall,dist_to_closest_place_marketplace,dist_to_closest_place_place_of_worship,dist_to_closest_place_public_building,dist_to_closest_place_school,dist_to_closest_place_supermarket,dist_to_closest_place_townhall,dist_to_closest_place_university,closest_levels,average_levels,max_levels,dist_to_closest_building,dist_to_closest_high_building,count_of_buildings,average_levels_of_closest_buildings
0,0,5478.0,8096,True,53.757824,87.159786,NaN,обл,4: город,0.0,53.794276,87.214405,Сибирский,547885.0,654000.0,Новокузнецк,2.0,57.0,5419.895573,0.000000,0.014118,0.014118,0.015074,0.021648,0.012992,2,14.0,amenity,0.0,1.0,13.0,0.0,1000.0,537.072,1000.000,104.119,parking,0.0,0.0,104.119,0.0,0.0,1000.0,1000.0,1000.000,1000.000,1000.000,1000.0,220.781,1000.000,1000.00,1000.000,1000.000,1000.000,244.639,1000.000,1000.0,1000.0,9.0,8.666667,12.0,30.190,30.190,18.0,7.0
1,1,3185.5,2570,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3185.5,2428,True,52.918598,158.514038,NaN,край,4: город,0.0,52.930428,158.405728,Дальневосточный,22905.0,684090.0,Вилючинск,1.0,9.0,7387.129365,0.006418,0.006418,0.108855,0.108855,0.109296,0.067968,3,18.0,shop,5.0,0.0,13.0,0.0,1000.0,1000.000,5.886,5.886,department_store,0.0,0.0,155.525,0.0,0.0,1000.0,1000.0,1000.000,5.886,98.378,1000.0,273.321,43.149,1000.00,1000.000,115.315,396.293,295.956,1000.000,1000.0,1000.0,2.0,3.612903,5.0,5.886,1000.000,31.0,3.0
3,3,3185.5,2799,True,55.533496,89.178148,NaN,край,4: город,0.0,55.538968,89.180156,Сибирский,38570.0,662310.0,Шарыпово,1.0,5.0,622.174851,0.005297,0.012029,0.016566,0.125603,0.689073,0.169714,1,18.0,amenity,5.0,0.0,13.0,0.0,1000.0,1000.000,233.627,9.952,school,0.0,0.0,334.479,2.0,2.0,1000.0,1000.0,272.963,1000.000,1000.000,1000.0,157.335,1000.000,1000.00,370.887,1000.000,1000.000,9.952,233.627,1000.0,1000.0,4.0,3.142857,5.0,9.952,1000.000,7.0,2.5
4,4,1942.0,250,True,55.179216,61.297729,NaN,обл,4: город,2.0,55.160366,61.400786,Уральский,1130273.0,454000.0,Челябинск,3.0,105.0,6880.126968,0.000000,0.000000,0.005201,0.006717,0.006717,0.003727,4,25.0,amenity,5.0,0.0,20.0,0.0,1000.0,1000.000,127.790,115.345,parking,0.0,0.0,115.345,1.0,0.0,1000.0,1000.0,127.790,448.813,1000.000,1000.0,266.739,1000.000,195.98,1000.000,1000.000,1000.000,208.316,1000.000,1000.0,1000.0,9.0,7.692308,19.0,28.489,28.489,26.0,5.0
